In [1]:
cd ..

C:\Users\User\Documents\SURF 2021\RL RNN SURF\src


We first define the input data. The input data is has the following structure: We have a group of subjects (say S1 and S2) and each subject has completed the decision-making task multiple times (multiple blocks), say each subject has completed the task two times in this example, i.e., we have two blocks of data for each subject. The data within each block is a ditonary containing three numpy arrays: 'action', 'state', 'reward'. 

'action' containig the actions taken by the subject on each trial and it should be a non-zero integer or -1. If the action is -1 it will coded by a zero vector and corresponds to no-action. Dimesionlity of 'action' is B x |T| in which |T| is the number of trials.

'state' contains the state of the environment each trial. Its dimesionlity is B x |T| x |S| in which |T| is the number of trials, and |S| is the lenght of state vector. 

'reward' contains the reward received after taking each actions. Its dimesionlity is B x |T| in which |T| is the number of trials.

For example, if subject S1 has completed 6 trials in the firt block and 4 trials in the second block and subject 2 has completed 5, 6 trials in the first and second blocks respectivly, then the data structure can look like this:

   

In [2]:
#  Data
import numpy as np
import scipy.io as spio
data = {}
for i in range(1,91):
    testData = spio.loadmat('genData_smG_rlG\sub_'+str(i)+'.mat')
    struct = testData['subData']
    data['S'+str(i)] = [
                {
                    'action': np.array([struct[0,0]['resp'].flatten()]),
                    'state': np.array([struct[0,0]['stimOffers']]),
                    'reward': np.array([struct[0,0]['outcomeRew'].flatten()]),
                    'id': 'S'+str(i),

                    'block': 0
                }

            ]


In [3]:
data

{'S1': [{'action': array([[1, 1, 2, 1, 1, 2, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 2, 2, 1, 2, 1,
           1, 2, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 1, 1, 2, 2, 2, 2, 2, 1, 2,
           1, 2, 1, 2, 2, 2, 1, 1, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 1, 2,
           2, 1, 1, 2, 2, 1, 1, 1, 1, 2, 2, 2, 2, 1, 2, 1, 2, 1, 1, 2, 1, 2,
           2, 2, 1, 2, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1, 2,
           2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1,
           2, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2, 2, 1, 2, 2, 2, 1, 1, 2, 1, 2,
           1, 1, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2,
           1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 2, 1, 1, 2, 2]], dtype=uint8),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           [2, 1],
           [3, 1],
           [2, 1],
           [1, 3],
           [2, 1],
           [2, 3],
           [3, 1],
           [3, 1],
           [1, 2],
           [2, 3],
           [1, 2],
           [1, 2],

In the above example, |A|=2 (there are two actions coded as 0 and 1), and |S|=2 (the state vector has two elements). For example, if there are three stimuli in the environment, they can be coded as [1, 0, 0], [0, 1, 0], [0, 0, 1] state vectors. In this case |S|=3.

In [4]:
from actionflow.rnn.lstm_beh import LSTMBeh
worker = LSTMBeh(a_size=2, s_size=2, n_cells=100)

c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
c:\users\user\documents\surf 2021\rl rnn surf\venv\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
c:\users\user\documents\surf

Instructions for updating:
This class is equivalent as tf.keras.layers.LSTMCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API


Finally, we train the model:

In [5]:
from actionflow.rnn.opt_beh import OptBEH
from actionflow.util.logger import LogFile

output_path = '../results/'
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, None,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None
                    )

2021-06-30 13:09:17,688 - DeepRL - DEBUG - version control: (None, None)
2021-06-30 13:09:17,689 - DeepRL - DEBUG - learning rate: 0.01
2021-06-30 13:09:17,689 - DeepRL - DEBUG - global iters: 50
2021-06-30 13:09:17,689 - DeepRL - DEBUG - training data-points: 90
2021-06-30 13:09:17,690 - DeepRL - DEBUG - test data-points: None


Instructions for updating:
Use tf.cast instead.


2021-06-30 13:09:18,110 - DeepRL - DEBUG - opt started...


INFO:tensorflow:../results/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-06-30 13:09:26,246 - DeepRL - DEBUG - global iter =    0 total obj: 185.8235
2021-06-30 13:09:33,757 - DeepRL - DEBUG - global iter =    1 total obj:  0.2410
2021-06-30 13:09:41,209 - DeepRL - DEBUG - global iter =    2 total obj:  0.1512
2021-06-30 13:09:48,601 - DeepRL - DEBUG - global iter =    3 total obj:  0.1032
2021-06-30 13:09:55,961 - DeepRL - DEBUG - global iter =    4 total obj:  0.0752
2021-06-30 13:10:03,434 - DeepRL - DEBUG - global iter =    5 total obj:  0.0575
2021-06-30 13:10:10,809 - DeepRL - DEBUG - global iter =    6 total obj:  0.0456
2021-06-30 13:10:18,140 - DeepRL - DEBUG - global iter =    7 total obj:  0.0371
2021-06-30 13:10:25,501 - DeepRL - DEBUG - global iter =    8 total obj:  0.0308
2021-06-30 13:10:32,805 - DeepRL - DEBUG - global iter =    9 total obj:  0.0260
2021-06-30 13:10:40,166 - DeepRL - DEBUG - global iter =   10 total obj:  0.0223
2021-06-30 13:10:47,483 - DeepRL - DEBUG - global iter =   11 total obj:  0.0193
2021-06-30 13:10:54,837 - D

INFO:tensorflow:../results/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [6]:
from actionflow.data.data_process import DataProcess
train_merged = DataProcess.merge_data(data)
train_merged

{'merged': [{'reward': array([[ 25,   0,  25, ..., -25, -25, -25],
          [ 25,   0,   0, ..., -25, -25, -25],
          [  0,   0,   0, ..., -25, -25, -25],
          ...,
          [ 25,   0,  25, ..., -25, -25,   0],
          [ 25,  25,   0, ...,   0, -25,   0],
          [ 25,   0,   0, ..., -25,   0,   0]], dtype=int16),
   'action': array([[1, 2, 1, ..., 2, 1, 1],
          [1, 1, 2, ..., 2, 1, 1],
          [1, 2, 1, ..., 1, 2, 1],
          ...,
          [2, 1, 1, ..., 1, 2, 1],
          [1, 2, 2, ..., 2, 2, 1],
          [1, 1, 2, ..., 1, 2, 2]], dtype=uint8),
   'state': array([[[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [1, 3]],
   
          [[3, 1],
           [3, 2],
           [3, 1],
           ...,
           [2, 1],
           [1, 3],
           [1, 3]],
   
          [[1, 3],
           [2, 1],
           [1, 2],
           ...,
           [1, 2],
           [3, 1],
           [2, 3]],
   
    

And then the merged data can be used for training the model as before. The test data can also be passed to the training method, in order to test the mmodel on the training data in regular intervals. Say the test data is as follows:

In [7]:
test_data = {}
for i in range(91,101):
    testData = spio.loadmat('genData_smG_rlG\sub_'+str(i)+'.mat')
    struct = testData['subData']
    test_data['S'+str(i)] = [
                {
                    'action': np.array([struct[0,0]['resp'].flatten()]),
                    'state': np.array([struct[0,0]['stimOffers']]),
                    'reward': np.array([struct[0,0]['outcomeRew'].flatten()]),
                    'id': 'S'+str(i),

                    'block': 0
                }

            ]


and we want to test the model every 10 iterations:

In [8]:
with LogFile(output_path, 'run.log'):
    OptBEH.optimise(worker, output_path, data, test_data,
                    learning_rate=0.01,
                    global_iters=50,
                    load_model_path=None,
                    test_period=10
                    )

2021-06-30 13:14:52,695 - DeepRL - DEBUG - version control: (None, None)
2021-06-30 13:14:52,696 - DeepRL - DEBUG - learning rate: 0.01
2021-06-30 13:14:52,696 - DeepRL - DEBUG - global iters: 50
2021-06-30 13:14:52,697 - DeepRL - DEBUG - training data-points: 90
2021-06-30 13:14:52,697 - DeepRL - DEBUG - test data-points: 10
2021-06-30 13:14:53,072 - DeepRL - DEBUG - opt started...
2021-06-30 13:14:53,074 - DeepRL - DEBUG - started testing...
2021-06-30 13:14:54,422 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/model-0/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-06-30 13:15:02,450 - DeepRL - DEBUG - global iter =    0 total obj: 177.2884
2021-06-30 13:15:09,500 - DeepRL - DEBUG - global iter =    1 total obj:  0.2275
2021-06-30 13:15:16,462 - DeepRL - DEBUG - global iter =    2 total obj:  0.1436
2021-06-30 13:15:23,474 - DeepRL - DEBUG - global iter =    3 total obj:  0.0983
2021-06-30 13:15:30,435 - DeepRL - DEBUG - global iter =    4 total obj:  0.0717
2021-06-30 13:15:37,406 - DeepRL - DEBUG - global iter =    5 total obj:  0.0548
2021-06-30 13:15:44,386 - DeepRL - DEBUG - global iter =    6 total obj:  0.0434
2021-06-30 13:15:51,372 - DeepRL - DEBUG - global iter =    7 total obj:  0.0353
2021-06-30 13:15:58,325 - DeepRL - DEBUG - global iter =    8 total obj:  0.0293
2021-06-30 13:16:05,294 - DeepRL - DEBUG - global iter =    9 total obj:  0.0247
2021-06-30 13:16:05,294 - DeepRL - DEBUG - started testing...
2021-06-30 13:16:06,644 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/model-10/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-06-30 13:16:14,328 - DeepRL - DEBUG - global iter =   10 total obj:  0.0211
2021-06-30 13:16:21,405 - DeepRL - DEBUG - global iter =   11 total obj:  0.0183
2021-06-30 13:16:28,529 - DeepRL - DEBUG - global iter =   12 total obj:  0.0160
2021-06-30 13:16:35,630 - DeepRL - DEBUG - global iter =   13 total obj:  0.0141
2021-06-30 13:16:42,596 - DeepRL - DEBUG - global iter =   14 total obj:  0.0125
2021-06-30 13:16:49,624 - DeepRL - DEBUG - global iter =   15 total obj:  0.0112
2021-06-30 13:16:56,685 - DeepRL - DEBUG - global iter =   16 total obj:  0.0100
2021-06-30 13:17:03,800 - DeepRL - DEBUG - global iter =   17 total obj:  0.0090
2021-06-30 13:17:10,867 - DeepRL - DEBUG - global iter =   18 total obj:  0.0082
2021-06-30 13:17:17,887 - DeepRL - DEBUG - global iter =   19 total obj:  0.0075
2021-06-30 13:17:17,887 - DeepRL - DEBUG - started testing...
2021-06-30 13:17:19,262 - DeepRL - DEBUG - finished testing.


...

INFO:tensorflow:../results/model-30/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-06-30 13:18:39,786 - DeepRL - DEBUG - global iter =   30 total obj:  0.0031
2021-06-30 13:18:46,988 - DeepRL - DEBUG - global iter =   31 total obj:  0.0029
2021-06-30 13:18:54,186 - DeepRL - DEBUG - global iter =   32 total obj:  0.0027
2021-06-30 13:19:01,294 - DeepRL - DEBUG - global iter =   33 total obj:  0.0026
2021-06-30 13:19:08,421 - DeepRL - DEBUG - global iter =   34 total obj:  0.0024
2021-06-30 13:19:15,543 - DeepRL - DEBUG - global iter =   35 total obj:  0.0023
2021-06-30 13:19:22,703 - DeepRL - DEBUG - global iter =   36 total obj:  0.0021
2021-06-30 13:19:29,842 - DeepRL - DEBUG - global iter =   37 total obj:  0.0020
2021-06-30 13:19:36,938 - DeepRL - DEBUG - global iter =   38 total obj:  0.0019
2021-06-30 13:19:44,081 - DeepRL - DEBUG - global iter =   39 total obj:  0.0018
2021-06-30 13:19:44,081 - DeepRL - DEBUG - started testing...
2021-06-30 13:19:45,513 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/model-40/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


2021-06-30 13:19:53,405 - DeepRL - DEBUG - global iter =   40 total obj:  0.0017
2021-06-30 13:20:00,607 - DeepRL - DEBUG - global iter =   41 total obj:  0.0016
2021-06-30 13:20:07,785 - DeepRL - DEBUG - global iter =   42 total obj:  0.0015
2021-06-30 13:20:14,927 - DeepRL - DEBUG - global iter =   43 total obj:  0.0014
2021-06-30 13:20:22,147 - DeepRL - DEBUG - global iter =   44 total obj:  0.0013
2021-06-30 13:20:29,292 - DeepRL - DEBUG - global iter =   45 total obj:  0.0013
2021-06-30 13:20:36,458 - DeepRL - DEBUG - global iter =   46 total obj:  0.0012
2021-06-30 13:20:43,590 - DeepRL - DEBUG - global iter =   47 total obj:  0.0011
2021-06-30 13:20:50,754 - DeepRL - DEBUG - global iter =   48 total obj:  0.0011
2021-06-30 13:20:57,894 - DeepRL - DEBUG - global iter =   49 total obj:  0.0010
2021-06-30 13:20:57,895 - DeepRL - DEBUG - opt finished.
2021-06-30 13:20:57,895 - DeepRL - DEBUG - started testing...
2021-06-30 13:20:59,324 - DeepRL - DEBUG - finished testing.


INFO:tensorflow:../results/model-final/model.cptk is not in all_model_checkpoint_paths. Manually adding it.


In [9]:
test_data["S100"]

[{'action': array([[1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 2, 2, 1, 1,
          2, 1, 2, 1, 1, 2, 1, 2, 1, 1, 2, 2, 2, 1, 1, 1, 1, 1, 2, 1, 2, 2,
          1, 1, 1, 2, 2, 1, 2, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1, 2, 1, 1, 1,
          2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 2, 2, 2, 1, 1, 2, 2, 2, 2,
          1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 1, 2, 1, 2, 1, 1, 2, 2, 1, 1, 2, 1,
          2, 1, 1, 1, 2, 2, 1, 1, 1, 2, 2, 1, 2, 2, 1, 1, 1, 1, 1, 1, 1, 2,
          2, 1, 1, 2, 1, 2, 2, 1, 1, 1, 1, 2, 1, 2, 2, 2, 2, 2, 1, 1, 1, 2,
          2, 1, 1, 2, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2, 2, 1, 2, 2, 2, 1, 1, 2,
          2, 2, 2, 1, 1, 1, 2, 1, 2, 1, 2, 2, 1, 1, 1, 2]], dtype=uint8),
  'state': array([[[3, 2],
          [1, 3],
          [1, 3],
          [1, 2],
          [2, 3],
          [1, 3],
          [1, 2],
          [3, 2],
          [1, 2],
          [2, 1],
          [2, 3],
          [3, 2],
          [3, 1],
          [2, 3],
          [3, 2],
          [2, 1],
          [